In this homework, you'll explore perplexity as a measure of evaluation for language models.

In [8]:
from nltk import word_tokenize, sent_tokenize
from collections import Counter
import copy
import numpy as np

In [9]:
def read_file(filename):
    sequences=[]
    with open(filename) as file:
        data=file.read()
        sents=sent_tokenize(data)
        for sent in sents:
            tokens=word_tokenize(sent)
            sequences.append(tokens)
            
    return sequences

In [10]:
# Read data from file and tokenize them into sequences comprised of tokens.

# Pride and Prejudice (Jane Austen)
sequences=read_file("../data/stylometry/1342_pride_and_prejudice.txt")

max_sequences=10000

In [11]:
class NgramModel():

    def __init__(self, sequences, order):
        
        # For this exercise we're going to encode the LM as a sparse dictionary (training less storage for more compute)
        # We'll store the LM as a dictionary with the conditioning context as keys; each value is a 
        # Counter object that keeps track of the number of times we see a word following that context.
        
        self.counts={}
        
        # Markov order (order 1 = conditioning on previous 1 word; order 2 = previous 2 words, etc.)
        self.order=order
        
        vocab={"[END]":0}
                
        for s_idx, tokens in enumerate(sequences):
            # We'll add [START] and [END] tokens to encode the beginning/end of sentences
            token_copy=copy.deepcopy(tokens)
            for i in range(order):
                token_copy.insert(0, "[START]")
            token_copy.append("[END]")
            
        
            for i in range(order, len(token_copy)):
                context=" ".join(token_copy[i-order:i])
                word=token_copy[i]
                
                if word not in vocab:
                    vocab[word]=len(vocab)
                
                # For just the first sentence, print the conditioning context + word
                if s_idx == 0:
                    print("Context: %s Next: %s" % (context.ljust(50), word))
                    
                if context not in self.counts:
                    self.counts[context]=Counter()
                self.counts[context][word]+=1
                


    def sample(self, context):

        total=sum(self.counts[context].values())
        
        dist=[]
        vocab=[]

        # Create a probability distribution for each conditioning context, over the vocab that we've observed it with.
        for idx, word in enumerate(self.counts[context]):
            prob=self.counts[context][word]/total
            dist.append(prob)
            vocab.append(word)

        index=np.argmax(np.random.multinomial(1, pvals=dist))
        return vocab[index]
        
    def generate_sequence(self):
        generated=["[START]"]*(self.order)
        word=None
        while word != "[END]":
            context=' '.join(generated[-self.order:] if self.order > 0 else "")
            word=self.sample(context)
            print(word)
            generated.append(word)
    
    

Let's create some language models of different orders from *Pride and Prejudice*.

In [13]:
ngram0=NgramModel(sequences[:max_sequences], order=0)

Context:                                                    Next: Chapter
Context:                                                    Next: 1
Context:                                                    Next: It
Context:                                                    Next: is
Context:                                                    Next: a
Context:                                                    Next: truth
Context:                                                    Next: universally
Context:                                                    Next: acknowledged
Context:                                                    Next: ,
Context:                                                    Next: that
Context:                                                    Next: a
Context:                                                    Next: single
Context:                                                    Next: man
Context:                                                    Next: in
Cont

In [6]:
ngram1=NgramModel(sequences[:max_sequences], order=1)

Context: [START]                                            Next: Chapter
Context: Chapter                                            Next: 1
Context: 1                                                  Next: It
Context: It                                                 Next: is
Context: is                                                 Next: a
Context: a                                                  Next: truth
Context: truth                                              Next: universally
Context: universally                                        Next: acknowledged
Context: acknowledged                                       Next: ,
Context: ,                                                  Next: that
Context: that                                               Next: a
Context: a                                                  Next: single
Context: single                                             Next: man
Context: man                                                Next: in
Cont

In [59]:
ngram2=NgramModel(sequences[:max_sequences], order=2)

Context: [START] [START]                                    Next: Chapter
Context: [START] Chapter                                    Next: 1
Context: Chapter 1                                          Next: It
Context: 1 It                                               Next: is
Context: It is                                              Next: a
Context: is a                                               Next: truth
Context: a truth                                            Next: universally
Context: truth universally                                  Next: acknowledged
Context: universally acknowledged                           Next: ,
Context: acknowledged ,                                     Next: that
Context: , that                                             Next: a
Context: that a                                             Next: single
Context: a single                                           Next: man
Context: single man                                         Next: in
Cont

**Q1.** Create a `perplexity` function that can takes two arguments: a.) a model of *any* ngram order (from the class above); and b.) a sequence to calculate perplexity for.  You'll recall from class that perplexity under a particular language model for sequence $w$ is given by the following equation:

$$
\textrm{perplexity}_{model}(w) = \exp\left(-{1 \over N} \sum_{i=1}^N \log P_{model}(w_i) \right)
$$

$P_{model}(w_i)$ calculates the probability of token $w_i$ using whatever assumptions that model makes -- for a bigram model (order 1), this is $P(w_i \mid w_{i-1})$, for a trigram model (order 2), this is $P(w_i \mid w_{i-2}, w_{i-1})$, etc.  Two things to note:

* When calculating the probability of the first word(s), be sure to get the conditioning context right.  The conditioning context for the first word in a trigram model, for example, is $P(w_i \mid$ [START] [START]$)$.
* Perplexity is only calculated for the words in the actual sequence.  We don't include the $P$([START]) or $P$([END]) in the perplexity calculuation.

In [66]:
def perplexity(model, tokens):
    total_log_prob = 0 
    total_tokens = 0  

    for i in range(model.order, len(tokens)):
        # Iterate through the tokens, starting from the 'order' index (context size) to exclude [START].
        context = ' '.join(tokens[i - model.order:i])  # Extract the context based on the model's order.
        current_token = tokens[i]  # Get the current token.

        if context in model.counts:
            # Calculate the probability of the current token given the context.
            context_counts = model.counts[context]
            token_count = context_counts.get(current_token, 0)

            if token_count > 0:
                # Avoid division by zero and calculate the probability.
                token_prob = token_count / sum(context_counts.values())

                # Add the logarithm of the probability to the total log probability.
                total_log_prob -= np.log(token_prob)
                total_tokens += 1  # Increment the token count.

    # Calculate perplexity as the exponential of the average log probability.
    perplexity = np.exp(total_log_prob / total_tokens)

    return (perplexity)  # Return the perplexity value.
   

**Q2**. Execute that perplexity function for the following language models:

In [67]:
perplexity(ngram0, word_tokenize("She was a great friend of Mr. Bingley."))

202.22631978521557

In [68]:
perplexity(ngram1, word_tokenize("She was a great friend of Mr. Bingley."))

24.554084030184423

In [69]:
perplexity(ngram2, word_tokenize("She was a great friend of Mr. Bingley."))

10.521370397120716

**Q3.** What is the perplexity of "She was a really great friend of Mr. Bingley." in the trigram language model trained above?  Explain in 100 words that behavior is expected (and correct) given how this language model and data.

In [58]:
perplexity(ngram2, word_tokenize("She was a really great friend of Mr. Bingley."))

5

**answer:** The perplexity score of this sentence is ([INSERT]). This makes sense because 1) as the markov order assumption increases, perp goes down because the model is better fit to the data. 2)This is correct because we are using the entire corpus?